# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
dataGroup = data.groupby(['CustomerID', 'ProductName']).sum().reset_index(level = 'ProductName')
dataGroup.drop(['SalesID','ProductID'], axis=1).head()

,ProductName,Quantity
CustomerID,,
33,Apricots - Dried,1
33,Assorted Desserts,1
33,Bandage - Flexible Neon,1
33,"Bar Mix - Pina Colada, 355 Ml",1
33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
matrix = pd.pivot_table(dataGroup, values ='Quantity', index = 'ProductName', columns ='CustomerID',\
                        aggfunc='sum',fill_value = 0)
matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
sim_matrix = squareform(pdist(matrix.T, metric='euclidean'))

distances_euclidean = pd.DataFrame(1/(1 + sim_matrix), 
                         index=matrix.columns, columns=matrix.columns)

distances_euclidean.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
#Extraigo la serie de la primera columna y elimino la primera posición que es 1, porque se compara con sí mismo.
#es decir para el cliente con ID 33, y aplico sobre esta serie un sort_values y luego me quedo con el top5:

similarities = distances_euclidean.loc[33]
top5_similar33 = pd.DataFrame(similarities.sort_values(ascending=False)[1:6])
display(top5_similar33)

top5_similar = list(top5_similar33.index)

top5_similar

,33
CustomerID,
264,0.087047
3535,0.087047
3317,0.087047
2503,0.085983
3305,0.085638


[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
top5_similar = list(top5_similar33.index)

top5_similar
top5_customer=dataGroup.loc[top5_similar]
top5_customer.head()

,ProductName,SalesID,ProductID,Quantity
CustomerID,,,,
264,Apricots - Halves,3730301,278,1
264,Apricots Fresh,5035449,158,1
264,Bacardi Breezer - Tropical,756802,49,1
264,Bagel - Plain,6728381,262,1
264,Banana - Leaves,6298459,271,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
productGroup = top5_customer.groupby ('ProductName').agg ({'Quantity':'sum'}).\
                sort_values(by = ['Quantity'],ascending = False)

productGroup.head()


,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
#Creo una lista de los productos que más compran los clientes con similitudes al 33 ya ordenados:

productGroupList = list(productGroup.index)

#Creo una lista de los productos que no ha comprado el cliente 33 a partir de la pivot_table:

customer33 = pd.DataFrame(matrix[33]).reset_index()
nonPurchased33 = list(customer33[customer33[33]== 0]['ProductName'])

#Compruebo qué productos de la lista 'productGroupList' no los ha comprado aún el cliente 33, 
#hasta llegar a 5 productos que serán la recomendación:

recommended33 = [e for e in productGroupList if e in nonPurchased33]

recommended33[:5]

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
#Creo lista de los clientes (no los incluyo todos para que se ejecute sin problemas):
allCustomers = [e for e in matrix.columns[:30]]

#Creo una función para iterar sobre los clientes de la lista:
def recommender (lista, distances):
    #Creo diccionario donde guardar para cada cliente las recomendaciones:
    dictRecommended = {}

    for e in lista:
        top5_similar = list(pd.DataFrame(distances.loc[e].sort_values(ascending=False)[0:6]).index[1:])
        top5_customer=dataGroup.loc[top5_similar]

        productGroup = top5_customer.groupby ('ProductName').agg ({'Quantity':'sum'}).\
                    sort_values(by = ['Quantity'],ascending = False)

        productGroupList = list(productGroup.index)
        customer = pd.DataFrame(matrix[e]).reset_index()
        nonPurchased = list(customer[customer[e]== 0]['ProductName'])

        recommended = [e for e in productGroupList if e in nonPurchased]
        dictRecommended [e] = recommended[:5]
        
    return dictRecommended


In [12]:
dictRecommender_euclidean = recommender (allCustomers,distances_euclidean)
print(dictRecommender_euclidean)

{33: ['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley', 'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein', 'Chicken - Soup Base'], 200: ['Soup - Campbells Bean Medley', 'Muffin - Carrot Individual Wrap', 'Bay Leaf', 'Pork - Kidney', 'Wanton Wrap'], 264: ['Soupfoamcont12oz 112con', 'Wine - Two Oceans Cabernet', 'Bread - Italian Roll With Herbs', 'Veal - Inside, Choice', 'Fish - Scallops, Cold Smoked'], 356: ['Veal - Inside, Choice', 'Lamb - Ground', 'Pomello', 'Butter - Unsalted', 'Tart Shells - Sweet, 4'], 412: ['Olive - Spread Tapenade', 'Sprouts - Baby Pea Tendrils', 'Wine - Blue Nun Qualitatswein', 'Pepper - Black, Whole', 'Soup - Campbells Bean Medley'], 464: ['Butter - Unsalted', 'Sauce - Gravy, Au Jus, Mix', 'Pepper - Black, Whole', 'Peas - Pigeon, Dry', 'Bar - Granola Trail Mix Fruit Nut'], 477: ['Olive - Spread Tapenade', 'Sprouts - Baby Pea Tendrils', 'Wine - Blue Nun Qualitatswein', 'Veal - Inside, Choice', 'Pepper - Black, Whole'], 639: ['Cod - Black Whole 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [14]:
tab_recommended_euclidean = pd.DataFrame(dictRecommender_euclidean)
tab_recommended_euclidean

,33,200,264,356,412,464,477,639,649,669,...,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503
0,Butter - Unsalted,Soup - Campbells Bean Medley,Soupfoamcont12oz 112con,"Veal - Inside, Choice",Olive - Spread Tapenade,Butter - Unsalted,Olive - Spread Tapenade,Cod - Black Whole Fillet,Bar - Granola Trail Mix Fruit Nut,"Veal - Inside, Choice",...,Sherry - Dry,"Pepper - Black, Whole",Soupfoamcont12oz 112con,"Tart Shells - Sweet, 4",Bandage - Fexible 1x3,Butter - Unsalted,Wine - Blue Nun Qualitatswein,Sausage - Breakfast,Knife Plastic - White,"Sole - Dover, Whole, Fresh"
1,Wine - Ej Gallo Sierra Valley,Muffin - Carrot Individual Wrap,Wine - Two Oceans Cabernet,Lamb - Ground,Sprouts - Baby Pea Tendrils,"Sauce - Gravy, Au Jus, Mix",Sprouts - Baby Pea Tendrils,Wine - Ej Gallo Sierra Valley,Wine - Magnotta - Belpaese,Sardines,...,Bandage - Fexible 1x3,Wine - Blue Nun Qualitatswein,"Veal - Inside, Choice",Muffin - Carrot Individual Wrap,Yoghurt Tubes,"Sauce - Gravy, Au Jus, Mix",Sardines,Bandage - Fexible 1x3,Butter - Unsalted,Whmis - Spray Bottle Trigger
2,Soup - Campbells Bean Medley,Bay Leaf,Bread - Italian Roll With Herbs,Pomello,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Yoghurt Tubes,Appetizer - Mushroom Tart,Wine - Ej Gallo Sierra Valley,...,Extract - Lemon,Soup - Campbells Bean Medley,Knife Plastic - White,Soup - Campbells Bean Medley,Wine - Chardonnay South,"Pepper - Black, Whole","Water, Tap",Beef - Texas Style Burger,"Beets - Candy Cane, Organic",Foam Dinner Plate
3,Wine - Blue Nun Qualitatswein,Pork - Kidney,"Veal - Inside, Choice",Butter - Unsalted,"Pepper - Black, Whole","Peas - Pigeon, Dry","Veal - Inside, Choice","Pepper - Black, Whole","Peas - Pigeon, Dry","Pepper - Black, Whole",...,"Fish - Scallops, Cold Smoked",Knife Plastic - White,Butter - Unsalted,"Cheese - Brie, Triple Creme",Knife Plastic - White,"Peas - Pigeon, Dry",Beef Wellington,Butter - Unsalted,Whmis - Spray Bottle Trigger,Table Cloth 54x72 White
4,Chicken - Soup Base,Wanton Wrap,"Fish - Scallops, Cold Smoked","Tart Shells - Sweet, 4",Soup - Campbells Bean Medley,Bar - Granola Trail Mix Fruit Nut,"Pepper - Black, Whole",Soupfoamcont12oz 112con,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,...,Knife Plastic - White,Muffin Batt - Blueberry Passion,Soup - Campbells Bean Medley,Lamb - Ground,Sprouts - Baby Pea Tendrils,Bar - Granola Trail Mix Fruit Nut,Wine - Chardonnay South,"Wine - Magnotta, Merlot Sr Vqa",Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [15]:
sim_matrix_cityblock = squareform(pdist(matrix.T, metric='cityblock'))

distances_cityblock = pd.DataFrame(1/(1 + sim_matrix_cityblock), 
                         index=matrix.columns, columns=matrix.columns)

distances_cityblock.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.008000,0.009901,0.009174,0.008403,0.008929,0.007752,0.008850,0.008621,0.008696,...,0.000564,0.000596,0.000556,0.000674,0.000542,0.000569,0.000556,0.000554,0.000476,0.000534
200,0.008000,1.000000,0.008130,0.007874,0.006993,0.007576,0.007752,0.007752,0.008065,0.008130,...,0.000563,0.000593,0.000557,0.000675,0.000538,0.000567,0.000558,0.000553,0.000473,0.000531
264,0.009901,0.008130,1.000000,0.008547,0.008850,0.008475,0.007874,0.008403,0.008621,0.008264,...,0.000566,0.000596,0.000556,0.000678,0.000543,0.000576,0.000559,0.000557,0.000479,0.000533
356,0.009174,0.007874,0.008547,1.000000,0.008000,0.008197,0.007752,0.009174,0.008065,0.008130,...,0.000564,0.000595,0.000556,0.000676,0.000540,0.000571,0.000553,0.000556,0.000478,0.000534
412,0.008403,0.006993,0.008850,0.008000,1.000000,0.008065,0.007752,0.008850,0.007937,0.008264,...,0.000561,0.000599,0.000553,0.000674,0.000540,0.000570,0.000552,0.000555,0.000477,0.000536


In [16]:
dictRecommenderCityblock = recommender (allCustomers,distances_cityblock)

In [17]:
tab_recommended_cityblock = pd.DataFrame(dictRecommenderCityblock)
tab_recommended_cityblock

,33,200,264,356,412,464,477,639,649,669,...,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503
0,Chocolate - Dark,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Butter - Unsalted,Wine - Blue Nun Qualitatswein,Cheese - Mozzarella,"Pepper - Black, Whole",Yoghurt Tubes,Ice Cream Bar - Hageen Daz To,Butter - Unsalted,...,Wanton Wrap,"Pepper - Black, Whole",Soupfoamcont12oz 112con,Muffin - Carrot Individual Wrap,"Salsify, Organic",Ketchup - Tomato,Sardines,Sausage - Breakfast,Knife Plastic - White,Table Cloth 54x72 White
1,Wine - Charddonnay Errazuriz,Pomello,Black Currants,"Veal - Inside, Choice","Pepper - Black, Whole",Butter - Unsalted,Ketchup - Tomato,"Pepper - Black, Whole","Peas - Pigeon, Dry",Lamb - Ground,...,Sausage - Liver,Wine - Blue Nun Qualitatswein,"Veal - Inside, Choice",Soup - Campbells Bean Medley,Apricots - Dried,Butter - Unsalted,"Water, Tap",Pastry - Choclate Baked,Butter - Unsalted,"Pepper - Black, Whole"
2,Bay Leaf,Bay Leaf,Pork - Kidney,"Beets - Candy Cane, Organic",Butter - Unsalted,Knife Plastic - White,Butter - Unsalted,Pork - Kidney,Squid - Tubes / Tenticles 10/20,"Pepper - Black, Whole",...,Ice Cream Bar - Hageen Daz To,Soup - Campbells Bean Medley,Knife Plastic - White,"Pasta - Detalini, White, Fresh",Butter - Unsalted,"Pepper - Black, Whole",Wine - Chardonnay South,Butter - Unsalted,"Beets - Candy Cane, Organic","Sole - Dover, Whole, Fresh"
3,"Oranges - Navel, 72",Pork - Kidney,"Veal - Inside, Choice","Nut - Chestnuts, Whole",Table Cloth 81x81 White,Muffin Batt - Blueberry Passion,Olive - Spread Tapenade,Lamb - Ground,Chinese Foods - Chicken,Ice Cream Bar - Hageen Daz To,...,Pork - Kidney,Knife Plastic - White,Butter - Unsalted,"Cheese - Boursin, Garlic / Herbs","Lamb - Pieces, Diced",Fenngreek Seed,Wine - Ej Gallo Sierra Valley,Squid - Tubes / Tenticles 10/20,Whmis - Spray Bottle Trigger,Muffin - Carrot Individual Wrap
4,Wine - Ej Gallo Sierra Valley,Muffin - Carrot Individual Wrap,Bread - Italian Roll With Herbs,Lamb - Ground,Soup - Campbells Bean Medley,Sausage - Breakfast,"Peas - Pigeon, Dry",Butter - Unsalted,"Pepper - Black, Whole",Wine - Ej Gallo Sierra Valley,...,Soup - Campbells Bean Medley,Muffin Batt - Blueberry Passion,Soup - Campbells Bean Medley,"Tart Shells - Sweet, 4",Sea Bass - Whole,Phyllo Dough,Pork - Hock And Feet Attached,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,Scallops 60/80 Iqf


In [18]:
#Con la siguiente comparación podemos ver que al utilizar métricas diferentes las recomendaciones varían:

tab_recommended_cityblock == tab_recommended_euclidean


,33,200,264,356,412,464,477,639,649,669,...,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503
0,False,True,False,False,False,False,False,False,False,False,...,False,True,True,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,True,True,False,False,False,False,False,True,False
2,False,True,False,False,False,False,False,False,False,False,...,False,True,True,False,False,True,False,False,True,False
3,False,True,True,False,False,False,False,False,False,False,...,False,True,True,False,False,False,False,False,True,False
4,False,False,False,False,True,False,False,False,True,False,...,False,True,True,False,False,False,False,False,True,False
